# Test de plusieurs modèles avec pondération sur Houses prices
----

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore") 

In [2]:
train = pd.read_csv('files_csv/trainmodif.csv',index_col=0)
test = pd.read_csv('files_csv/testmodif.csv',index_col=0)

df = pd.read_csv('files_csv/test.csv')

On recupere les id du fichier Kaggle pour la soumission

In [3]:
test_ID = df['Id']
test_ID

0       1461
1       1462
2       1463
3       1464
4       1465
        ... 
1454    2915
1455    2916
1456    2917
1457    2918
1458    2919
Name: Id, Length: 1459, dtype: int64

Train et Test avec traitement des données , log1p sur SalePrice , robustScaler 

In [4]:
print(train.shape)
print(test.shape)

(1444, 219)
(1459, 218)


In [5]:
train.head()

,LotFrontage,LotArea,Street,Alley,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,...,MSSubClass_class2,MSSubClass_class3,MSSubClass_class4,MSSubClass_class5,MSSubClass_class6,MSSubClass_class7,MSSubClass_class8,MSSubClass_class9,MSSubClass_class13,SalePrice
0,-0.25,-0.252090,0.0,0.0,0.0,0.5,0.0,0.648649,0.243243,1.200613,...,0,0,0,0,1,0,0,0,0,12.247699
1,0.50,0.037091,0.0,0.0,0.0,0.0,3.0,0.064865,-0.486486,0.000000,...,0,0,0,0,0,0,0,0,0,12.109016
2,-0.10,0.452002,0.0,0.0,0.0,0.5,0.0,0.605405,0.216216,0.992343,...,0,0,0,0,1,0,0,0,0,12.317171
3,-0.50,0.024518,0.0,0.0,0.0,0.5,0.0,-1.254054,-0.648649,0.000000,...,0,0,0,0,0,1,0,0,0,11.849405
4,0.70,1.208902,0.0,0.0,0.0,1.0,0.0,0.583784,0.162162,2.143951,...,0,0,0,0,1,0,0,0,0,12.429220


In [6]:
test.head()

,LotFrontage,LotArea,Street,Alley,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,...,MSSubClass_class16,MSSubClass_class2,MSSubClass_class3,MSSubClass_class4,MSSubClass_class5,MSSubClass_class6,MSSubClass_class7,MSSubClass_class8,MSSubClass_class9,MSSubClass_class13
0,0.50,0.545546,0.0,0.0,0.0,-0.5,1.0,-0.259459,-0.891892,0.000000,...,0,0,0,0,0,0,0,0,0,0
1,0.55,1.210662,0.0,0.0,0.0,0.0,1.0,-0.324324,-0.972973,0.661562,...,0,0,0,0,0,0,0,0,0,0
2,0.20,1.100773,0.0,0.0,0.0,-0.5,0.0,0.518919,0.108108,0.000000,...,0,0,0,0,0,1,0,0,0,0
3,0.40,0.132143,0.0,0.0,0.0,0.0,1.0,0.540541,0.108108,0.122511,...,0,0,0,0,0,1,0,0,0,0
4,-1.35,-1.118376,0.0,0.0,0.0,1.0,0.0,0.410811,-0.054054,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [7]:
y_train = train['SalePrice'].values
train.drop(['SalePrice'], axis=1, inplace=True)

In [8]:
X_train = train.values

In [9]:
X_test = test.values

#### Scaler
Scaler le prix aussi ? 

In [10]:
from sklearn.preprocessing import RobustScaler

#### différents modeles

In [11]:
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
import numpy as np

In [12]:
# Cross-validation with k-folds
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [13]:
ridge = Ridge()
model_xgb = xgb.XGBRegressor()
model_lgb = lgb.LGBMRegressor()

In [14]:
# lasso = Lasso()
# ENet = ElasticNet()
# KRR = KernelRidge()
# ridge = Ridge()
# GBoost = GradientBoostingRegressor()
# model_xgb = xgb.XGBRegressor()
# model_lgb = lgb.LGBMRegressor()

In [15]:
score = rmsle_cv(ridge)
print("Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(model_lgb)
print("Model LGB score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Ridge score: 0.1174 (0.0047)

[11:19:55] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:19:56] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:19:56] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:19:57] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:19:58] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Xgboost score: 0.1211 (0.0052)

Model LG

In [16]:
# score = rmsle_cv(lasso)
# print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
# score = rmsle_cv(ENet)
# print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
# score = rmsle_cv(KRR)
# print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
# score = rmsle_cv(ridge)
# print("Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
# score = rmsle_cv(GBoost)
# print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
# score = rmsle_cv(model_xgb)
# print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
# score = rmsle_cv(model_lgb)
# print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [17]:
# rw = 0.1174
# xw = 0.1212
# Sum = 0.1174 + 0.1212

In [18]:
# 1 - ( rw / Sum ) 

In [19]:
# 1 - ( xw / Sum ) 

#### Ridge

In [20]:
ridge = Ridge()

param_grid = [
    {'solver' : ['svd'], 'alpha':[x for x in range(1,55,1)]}
    ]
grid_search = GridSearchCV(ridge,param_grid,cv=5, scoring='neg_mean_squared_error' ,return_train_score=True)

grid_search.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'solver': ['svd'], 'alpha': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [21]:
grid_search.best_estimator_

Ridge(alpha=20, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='svd', tol=0.001)

In [22]:
y_ridge = np.expm1(grid_search.best_estimator_.predict(X_test))

### Xgb

In [23]:
# model_xgb = xgb.XGBRegressor()

# param_grid = [
# #     {'n_estimators': [x for x in range(790,800,1)], 'max_depth':[2]}
#     {'n_estimators':[300]}
#     ]
# grid_search = GridSearchCV(model_xgb,param_grid,cv=5, scoring='neg_mean_squared_error' ,return_train_score=True)

# grid_search.fit(X_train,y_train)

#  param_grid = {'XGBoost__n_estimators': [200, 300,500, 800, 1000],
       #       'XGBoost__max_depth': [4],
       #       'XGBoost__learning_rate':[0.1],
       #       'XGBoost__min_child_weight' : [3],
       #       'XGBoost__gamma'            : [0.0],
       #       'XGBoost__colsample_bytree' : [0.5], 

#Aro Params
model_xgb = xgb.XGBRegressor(n_estimators=300 , max_depth=4, learning_rate=0.1, gamma=0.0, colsample_bytree=0.5)
model_xgb.fit(X_train,y_train)

[11:20:07] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.5, gamma=0.0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [24]:
grid_search.best_estimator_

Ridge(alpha=20, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='svd', tol=0.001)

In [25]:
y_xgb = np.expm1(grid_search.best_estimator_.predict(X_test))

### LGB Regressor

In [26]:
model_lgb = lgb.LGBMRegressor()

param_grid = [
    {'n_estimators': [x for x in range(2140,2161,1)], 'max_depth':[1]}
    ]
grid_search = GridSearchCV(model_lgb,param_grid,cv=5, scoring='neg_mean_squared_error' ,return_train_score=True)

grid_search.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_estimators': [2140, 2141, 2142, 2143, 2144, 2145, 2146, 2147, 2148, 2149, 2150, 2151, 2152, 2153, 2154, 2155, 2156, 2157, 2158, 2159, 2160], 'max_depth': [1]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [27]:
grid_search.best_estimator_

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=2154, n_jobs=-1, num_leaves=31, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [28]:
y_lgb = np.expm1(grid_search.best_estimator_.predict(X_test))

#### Prédictions

In [29]:
y_val_kaggle = y_ridge * 0.6 + y_xgb * 0.2 + y_lgb*0.2

In [30]:
y_val_kaggle

array([115968.27676804, 155596.82075029, 179883.14853257, ...,
       174623.81391897, 115694.22434761, 217539.26635566])

In [31]:
sub = pd.DataFrame()
sub['Id'] = test_ID
sub['SalePrice'] = y_val_kaggle
sub.to_csv('xgb_ridge_lgb.csv',index=False)